#   Struct

###  Andrea Iskander Belkhir id : 511089

###  Beyza Özdemir id : 576145

In [1]:
using BenchmarkTools
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using StaticArrays 

#  Traversal(CTM::matrix,stack,obj,scene=[])

trasform the gerarchic in a unique coordinate system(world coordinate), the body of the obj is deepth-first searched and during the visit is used and updated the ctm matrix (by moltiplication the vertices(self multiplication))
during the visit we use the stack by pushing and poping the current ctm

In [2]:
square = Lar.cuboid([1,1])

([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [3]:
structure1 = Lar.Struct([square])

LinearAlgebraicRepresentation.Struct([([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])], [[0.0; 0.0], [1.0; 1.0]], "14118103471886415402", 2, "feature")

In [4]:
table = Lar.apply(Lar.t(-0.5,-0.5),square)

([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])

In [5]:
structure2 = Lar.Struct([repeat([table,Lar.r(pi/2)],outer=2)...])

LinearAlgebraicRepresentation.Struct(Any[([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0], ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0]], [[-0.5; -0.5], [0.5; 0.5]], "7186358468044932211", 2, "feature")

In [6]:
Id = Matrix{Float64}(LinearAlgebra.I, 3, 3)

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

In [7]:
function traversal(CTM::Matrix, stack, obj, scene=[])
	for i = 1:length(obj.body)
		if isa(obj.body[i],Matrix)
			CTM = CTM*obj.body[i]
		elseif (isa(obj.body[i],Tuple) || isa(obj.body[i],Array)) && (length(obj.body[i])>=2)
			l = Lar.apply(CTM, obj.body[i])
			push!(scene,l)
		elseif isa(obj.body[i],Lar.Struct)
			push!(stack,CTM)
			Lar.traversal(CTM,stack,obj.body[i],scene)
			CTM = pop!(stack)
		end
	end
	return scene
end

traversal (generic function with 2 methods)

In [8]:
@btime traversal(Id,[],structure1,[])

  1.330 μs (16 allocations: 1.14 KiB)


1-element Vector{Any}:
 ([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [9]:
@btime traversal(Id,[],structure2,[])

  3.638 μs (25 allocations: 2.23 KiB)


2-element Vector{Any}:
 ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])
 ([0.49999999999999994 -0.5 0.5 -0.49999999999999994; -0.5 -0.49999999999999994 0.49999999999999994 0.5], [[1, 2, 3, 4]])

In [10]:
@benchmark traversal(Id,[],structure2,[])

BenchmarkTools.Trial: 10000 samples with 8 evaluations.
 Range (min … max):  3.700 μs … 642.450 μs  ┊ GC (min … max): 0.00% … 98.62%
 Time  (median):     4.000 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.030 μs ±  11.045 μs  ┊ GC (mean ± σ):  3.69% ±  1.70%

  ▆█                                                           
  ██▆▃▄▃▃▃▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  3.7 μs          Histogram: frequency by time        11.1 μs <

 Memory estimate: 2.23 KiB, allocs estimate: 25.

# Improvment & changes

In [11]:
function traversal_imp(CTM::Matrix, stack, obj, scene)
	@inbounds for i in 1:length(obj.body)
		if (isa(obj.body[i],MMatrix) || isa(obj.body[i],Matrix))
			CTM = CTM*obj.body[i]
		elseif (isa(obj.body[i],Tuple) || isa(obj.body[i],Array)) && (length(obj.body[i])>=2)
			l = Lar.apply(CTM, obj.body[i])
			push!(scene,l)
		elseif isa(obj.body[i],Lar.Struct)
			push!(stack,CTM)
			Lar.traversal(CTM,stack,obj.body[i],scene)
			CTM = pop!(stack)
		end
	end
	return scene
end

traversal_imp (generic function with 1 method)

In [12]:
@btime traversal_imp(Id,[],structure2,[])

  3.800 μs (25 allocations: 2.23 KiB)


2-element Vector{Any}:
 ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])
 ([0.49999999999999994 -0.5 0.5 -0.49999999999999994; -0.5 -0.49999999999999994 0.49999999999999994 0.5], [[1, 2, 3, 4]])

In [13]:
@benchmark traversal_imp(Id,[],structure2,[])

BenchmarkTools.Trial: 10000 samples with 8 evaluations.
 Range (min … max):  3.788 μs … 794.225 μs  ┊ GC (min … max): 0.00% … 98.56%
 Time  (median):     4.125 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.693 μs ±  11.940 μs  ┊ GC (mean ± σ):  4.32% ±  1.70%

   █▆                                                          
  ▃██▇▆▅▄▃▃▂▃▃▃▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  3.79 μs         Histogram: frequency by time        9.66 μs <

 Memory estimate: 2.23 KiB, allocs estimate: 25.

# Paralelization

In [14]:
 function traversal_asy(CTM::Matrix, stack, obj, scene)
	@inbounds @async for i in 1:length(obj.body)
		if (isa(obj.body[i],MMatrix) || isa(obj.body[i],Matrix))
			CTM = CTM*obj.body[i]
		elseif (isa(obj.body[i],Tuple) || isa(obj.body[i],Array)) && (length(obj.body[i])>=2)
			l = Lar.apply(CTM, obj.body[i])
			push!(scene,l)
		elseif isa(obj.body[i],Lar.Struct)
			push!(stack,CTM)
			traversal(CTM,stack,obj.body[i],scene)
			CTM = pop!(stack)
		end
	end
	return scene
end

traversal_asy (generic function with 1 method)

In [15]:
@benchmark traversal_asy(Id,[],structure2,[])

BenchmarkTools.Trial: 10000 samples with 197 evaluations.
 Range (min … max):  538.071 ns …  4.987 ms  ┊ GC (min … max):  0.00% … 99.97%
 Time  (median):     720.558 ns              ┊ GC (median):     0.00%
 Time  (mean ± σ):     2.096 μs ± 61.532 μs  ┊ GC (mean ± σ):  49.77% ±  2.00%

  ▆█▆▅▃▃▂▂▁▁▂▂▂▃▅▅▅▄▃▃▂▂▁▁▁▁▁ ▁▁  ▁ ▁▁ ▁                       ▂
  █████████████████████████████████████████▇▇▇▇▆▆▆▆▅▆▆▅▅▆▄▆▄▅▅ █
  538 ns        Histogram: log(frequency) by time      3.29 μs <

 Memory estimate: 704 bytes, allocs estimate: 8.

# when using more threads is slower then the async but have a better range for the max and min time

In [16]:
 function traversal_thead(CTM::Matrix, stack, obj, scene)
	@inbounds Threads.@spawn for i in 1:length(obj.body)
		if (isa(obj.body[i],MMatrix) || isa(obj.body[i],Matrix))
			CTM = CTM*obj.body[i]
		elseif (isa(obj.body[i],Tuple) || isa(obj.body[i],Array)) && (length(obj.body[i])>=2)
			l = Lar.apply(CTM, obj.body[i])
			push!(scene,l)
		elseif isa(obj.body[i],Lar.Struct)
			push!(stack,CTM)
			traversal(CTM,stack,obj.body[i],scene)
			CTM = pop!(stack)
		end
	end
	return scene
end

traversal_thead (generic function with 1 method)

In [17]:
@benchmark traversal_thead(Id,[],structure2,[])

BenchmarkTools.Trial: 9396 samples with 198 evaluations.
 Range (min … max):  291.414 ns …   8.468 ms  ┊ GC (min … max):  0.00% … 99.98%
 Time  (median):     826.263 ns               ┊ GC (median):     0.00%
 Time  (mean ± σ):     2.820 μs ± 104.965 μs  ┊ GC (mean ± σ):  63.23% ±  2.06%

          ▁▅▇█▅▁                                                 
  ▁▁▁▂▃▃▃▅██████▅▄▃▃▄▆▆▆▆▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁ ▂
  291 ns           Histogram: frequency by time         2.75 μs <

 Memory estimate: 832 bytes, allocs estimate: 9.